In [88]:
import numpy as np
A = [[1, 1, 2, 2], [2, 1, 1, 2], [2, 2, 1, 1], [1, 2, 2, 1]]
B = [[1, 1, 2, 2], [2, 2, 1, 1], [2, 2, 1, 1], [1, 2, 2, 1]]
C = np.array([
            [0, 0, 0, 0], 
            [0, 1, 0, 1], 
            [0, 0, 2, 0], 
            [0, 1, 0, 0]], dtype=np.int8)
D = np.array([
            [1, 1, 0, 0], 
            [0, 1, 1, 0], 
            [0, 0, 1, 1], 
            [1, 0, 0, 1]], dtype=np.int8)
E = np.array([  # Unique 
            [1, 2, 2, 1], 
            [2, 1, 1, 2], 
            [2, 0, 0, 0], 
            [1, 2, 0, 0]], dtype=np.int8)
F = np.array([  # Search
            [0, 0, 0, 0], 
            [0, 1, 0, 1], 
            [0, 0, 2, 0], 
            [0, 0, 0, 0]], dtype=np.int8)
# F = np.array([  # Passes validation
#             [2, 1, 1, 2], 
#             [1, 2, 2, 1], 
#             [2, 1, 1, 2], 
#             [1, 2, 2, 1]], dtype=np.int8)


GRID = C.copy()
dim = len(GRID)
dim_2 = dim // 2
FLIP = np.array([0, 2, 1])
sum_standard = sum

## Replacing numpy with jampy
Aim to be able to write
```python	
np.any(np.sum(grid == 1, axis=0) > dim_2)
list(zip(*np.where(grid == 0)))  # replaced by just where
np.unique(full_rows, axis=0)
grid.T
```

In [110]:
GRID = np.array(B).T
np.unique(GRID, axis=1)

GRID = np.array(B)
np.unique(GRID, axis=0)

array([[1, 1, 2, 2],
       [1, 2, 2, 1],
       [2, 2, 1, 1]])

In [106]:
GJID = array(B)
print(type(GJID))
# hash(tuple(A))
# [item for row in GJID.array for item in row]
# hash(tuple([item for row in GJID.array for item in row]))
unique(GJID, axis=1)

<class '__main__.jdarray'>


[[1, 1, 2, 2]
 [2, 2, 1, 1]
 [2, 2, 1, 1]
 [1, 2, 2, 1]]

In [30]:
_A = [x+y for x in range(3) for y in range(x)]

In [105]:
def array(array):
    return jdarray(array)

def ones(shape):
    array = jdarray(shape)
    array._ones(shape)
    return array

def zeros(shape):
    array = jdarray(shape)
    array._zeros(shape)
    return array

# Won't all and any always be 1d?
def all(array):
    if isinstance(array[0], list):  # 2d
        for row in array:
            for item in row:
                if not item:
                    return False
    else:  # 1d
        for item in array:
            if not item:
                return False
    return True

def any(array):
    if isinstance(array[0], list):  # 2d
        for row in array:
            for item in row:
                if item:
                    return True
    else:  # 1d
        for item in array:
            if item:
                return True
    return False

def sum(array, axis=None):
    return array.sum(axis)

def _where(array):
    return [(i, j) for i, row in enumerate(array) for j, item in enumerate(row) if item]

def where(array):
    """ Returns two lists of indices of non-zero elements.
    Use _where instead to get a list of tuples."""
    rows, cols = [], []
    for i, row in enumerate(array):
        for j, item in enumerate(row):
            if item:
                rows.append(i)
                cols.append(j)
    return jdarray(rows), jdarray(cols)

def unique(array, axis):
    """Returns a list of unique rows or columns"""
    # return list(set([item for row in array for item in row]))
    if axis == 0:
        # return set(array)
        return jdarray([array[i] for i in range(len(array)) if array[i] not in array[:i]])
    elif axis == 1:
        transposed = array.T
        return jdarray([array[i] for i in range(len(transposed)) if transposed[i] not in transposed[:i]])
        # return jdarray([array[:, i] for i in range(len(array[0])) if array[:, i] not in array[:, :i]])
    else:
        raise ValueError(f"unique axis must be 0 or 1, not {axis}")

class jdarray():
    """A ndarray clone that supports only 2d arrays, but a few places are written to support more dimensions.
    1d array is implemented too, for list > int stuff"""

    def __init__(self, shape):
        if isinstance(shape, int):
            self.shape = (shape, shape)
        elif isinstance(shape, tuple):
            self.shape = shape
        elif isinstance(shape, list):  # Broken 'shape' name
            if isinstance(shape[0], list):
                self.shape = (len(shape), len(shape[0]))
                self.array = shape
            else:
                self.shape = (len(shape), )
                self.array = shape
        else:
            raise TypeError(f"jdarray.__init__ shape must be int, tuple or list, not {type(shape)}")
    
    def sum(self, axis=None):
        if axis is None:
            return sum_standard([sum_standard(row) for row in self.array])
        elif axis == 0:
            # return [sum_standard(row) for row in zip(*self.array)]
            return jdarray([sum_standard(row) for row in zip(*self.array)])
        elif axis == 1:
            # return [sum_standard(row) for row in self.array]
            return jdarray([sum_standard(row) for row in self.array])
    
    def _ones(self, shape):
        if len(shape) == 1:
            return [1] * shape[0]
        self.array = [self._ones(shape[1:]) for _ in range(shape[0])]
            
    def _zeros(self, shape):
        if len(shape) == 1:
            return [0] * shape[0]
        self.array = [self._zeros(shape[1:]) for _ in range(shape[0])]
    
    @property
    def T(self):
        return jdarray([list(row) for row in zip(*self.array)])
    
    def __getitem__(self, key):
        if isinstance(key, int):
            return self.array[key]
        
        elif isinstance(key, tuple):
            if len(key) > len(self.shape):
                raise IndexError(f"jdarray.__getitem__ key {key} is out of range")
            return self.array[key[0]][key[1]]
        
        elif isinstance(key, slice):
            return self.array[key]

        elif isinstance(key, jdarray):
            return [self.array[i][j] for i, row in enumerate(key.array) for j, b in enumerate(row) if b]

        else:
            raise TypeError(f"jdarray.__getitem__ key must be int or tuple, not {type(key)}")
    
    def __setitem__(self, key, value):
        if isinstance(key, int):
            self.array[key] = value
        elif isinstance(key, tuple):
            if len(key) > len(self.shape):
                raise IndexError(f"jdarray.__getitem__ key {key} is out of range")
            self.array[key[0]][key[1]] = value
        else:
            raise TypeError(f"jdarray.__setitem__ key must be int or tuple, not {type(key)}")
        
    def __len__(self):
        return len(self.array)
    
    def __iter__(self):
        return iter(self.array)
    
    def __contains__(self, item):
        return item in self.array
    
    def __hash__(self):
        return hash(tuple([item for row in self.array for item in row]))
    
    def __mul__(self, integer):
        if isinstance(integer, int):
            self.array = self._mul(self.array, integer)
            return self
        else:
            raise TypeError(f"jdarray.__mul__ integer must be int, not {type(integer)}")
    
    def _mul(self, array, integer):
        if isinstance(array[0], int):  # No more nests
            return [integer * element for element in array]
        return [self._mul(entree, integer) for entree in array]
    
    def __repr__(self):
        # print('jdarray(' + str(self.shape) + ')')
        if len(self.shape) == 1:
            return f'array({str(self.array)})'
        string = "["
        for row in self.array:
            string += str(row) + '\n '
        return string[:-2] + "]"
    
    def __add__(self, axis=None):
        if axis is None:
            return sum_standard([sum_standard(row) for row in self.array])
        elif axis == 0:
            return [sum_standard(row) for row in zip(*self.array)]
        elif axis == 1:
            return [sum_standard(row) for row in self.array]
    
    def __eq__(self, other):
        if isinstance(other, int):
            array = self._eq(self.array, other)
            result = jdarray(self.shape)
            result.array = array
            return result
        else:
            raise TypeError(f"jdarray.__eq__ other must be int, not {type(other)}")
    
    def _eq(self, array, other):
        if isinstance(array[0], int):
            return [element == other for element in array]
        return [self._eq(entree, other) for entree in array]

    def __gt__(self, other):  # 
        if isinstance(other, int):
            array = self._gt(self.array, other)
            result = jdarray(self.shape)
            result.array = array
            return result
        else:
            raise TypeError(f"jdarray.__gt__ other must be int, not {type(other)}")
    
    def _gt(self, array, other):
        if isinstance(array[0], int):
            return [element > other for element in array]
        return [self._gt(entree, other) for entree in array]
    
    def __lt__(self, other):
        if isinstance(other, int):
            array = self._lt(self.array, other)
            result = jdarray(self.shape)
            result.array = array
            return result
        else:
            raise TypeError(f"jdarray.__lt__ other must be int, not {type(other)}")
        
    def _lt(self, array, other):
        if isinstance(array[0], int):
            return [element < other for element in array]
        return [self._lt(entree, other) for entree in array]
    
print(array(A))

[[1, 1, 2, 2]
 [2, 1, 1, 2]
 [2, 2, 1, 1]
 [1, 2, 2, 1]]


In [26]:
# A = [0 for x in range(y) for y in range(10)]
A = []
for x in range(10):
    for y in range(x):
        A.append(0)
print(A)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [42]:
# Instancing
print("Instancing")
_shape = (2, 3)
narray = np.zeros(_shape)
narray[1,1] = 5
print(narray)

jarray = zeros(_shape)
jarray[1, 1] = 5
print(jarray)

print("\nMultiplication")
jarray = ones(_shape)
print(jarray, 'before')
jarray *= 4
print(jarray, 'after')

print("\nSumming")
_shape = (2, 3)
narray = np.ones(_shape)
jarray = ones(_shape)
print(f'Total: numpy: {narray.sum()}\t Jarray: {jarray.sum()}')
print(f'Rows : numpy: {narray.sum(axis=1)}\t Jarray: {jarray.sum(axis=1)}')
print(f'Cols : numpy: {narray.sum(axis=0)}\t Jarray: {jarray.sum(axis=0)}')

print('\nEqual')
_shape = (2, 3)
narray = np.ones(_shape)
jarray = ones(_shape)
narray[1,1] = 0
jarray[1,1] = 0
print(narray[narray == 1])
print(jarray[jarray == 1])

print('\nWhere')
GRID = np.array(A)
print(np.where(GRID == 1))
print(list(zip(*np.where(GRID == 1))))

GJID = array(A)
print(where(GJID == 1))
print(list(zip(*where(GJID == 1))))
# print("\nGreater than")
# print('\nUnique')

Instancing
[[0. 0. 0.]
 [0. 5. 0.]]
[[0, 0, 0]
 [0, 5, 0]]

Multiplication
[[1, 1, 1]
 [1, 1, 1]] before
[[4, 4, 4]
 [4, 4, 4]] after

Summing
Total: numpy: 6.0	 Jarray: 6
Rows : numpy: [3. 3.]	 Jarray: array([3, 3])
Cols : numpy: [2. 2. 2.]	 Jarray: array([2, 2, 2])

Equal
[1. 1. 1. 1. 1.]
[1, 1, 1, 1, 1]

Where
(array([0, 0, 1, 1, 2, 2, 3, 3], dtype=int64), array([0, 1, 1, 2, 2, 3, 0, 3], dtype=int64))
[(0, 0), (0, 1), (1, 1), (1, 2), (2, 2), (2, 3), (3, 0), (3, 3)]
(array([0, 0, 1, 1, 2, 2, 3, 3]), array([0, 1, 1, 2, 2, 3, 0, 3]))
[(0, 0), (0, 1), (1, 1), (1, 2), (2, 2), (2, 3), (3, 0), (3, 3)]


In [38]:
GRID = np.array(A)
print(GRID == 1)
print(np.sum(GRID == 1, axis=0))
print(np.sum(GRID == 1, axis=0) > dim_2)
print(np.any(np.sum(GRID == 1, axis=0) > dim_2))
GJID = array(A)
print(GJID == 1)
print(sum(GJID == 1, axis=0))
print(sum(GJID == 1, axis=0) > dim_2)
print(any(sum(GJID == 1, axis=0) > dim_2))

[[ True  True False False]
 [False  True  True False]
 [False False  True  True]
 [ True False False  True]]
[2 2 2 2]
[False False False False]
False
[[True, True, False, False]
 [False, True, True, False]
 [False, False, True, True]
 [True, False, False, True]]
array([2, 2, 2, 2])
array([False, False, False, False])
False


In [37]:
GRID = np.array(A)
print(np.where(GRID == 1))
print(list(zip(*np.where(GRID == 1))))

GJID = array(A)
print(where(GJID == 1))
print(list(zip(*where(GJID == 1))))

(array([0, 0, 1, 1, 2, 2, 3, 3], dtype=int64), array([0, 1, 1, 2, 2, 3, 0, 3], dtype=int64))
[(0, 0), (0, 1), (1, 1), (1, 2), (2, 2), (2, 3), (3, 0), (3, 3)]
(array([0, 0, 1, 1, 2, 2, 3, 3]), array([0, 1, 1, 2, 2, 3, 0, 3]))
[(0, 0), (0, 1), (1, 1), (1, 2), (2, 2), (2, 3), (3, 0), (3, 3)]


## Rules

In [3]:
def valid_grid(grid, verbose=False):
    """Check that grid is valid."""
    dim = len(grid)
    dim_2 = dim // 2
    
    # Equal count
    if np.any(np.sum(grid == 1, axis=1) > dim_2) or np.any(np.sum(grid == 2, axis=1) > dim_2):
        if verbose:
            print("Unequal count - rows")
        return False
    if np.any(np.sum(grid == 1, axis=0) > dim_2) or np.any(np.sum(grid == 2, axis=0) > dim_2):
        if verbose:
            print("Unequal count - cols")
        return False
    
    # Subsequent (3 in a row)
    for r in range(dim):
        for c in range(dim):
            if c < dim-2 and grid[r, c] == grid[r, c+1] == grid[r, c+2] != 0:
                if verbose:
                    print("Subsequent - rows")
                return False
            if r < dim-2 and grid[r, c] == grid[r+1, c] == grid[r+2, c] != 0:
                if verbose:
                    print("Subsequent - cols")
                return False
    
    # Unique
    full_rows = grid[np.where(grid.all(axis=1))[0]]
    if len(full_rows) != len(np.unique(full_rows, axis=0)):
        if verbose:
            print("Duplicate - rows")
        return False
    
    full_cols = grid.T[np.where(grid.all(axis=0))[0]]
    if len(full_cols) != len(np.unique(full_cols, axis=0)):
        if verbose:
            print("Duplicate - cols")
        return False
    
    return True

## Solver

In [1]:
class Solver():
    """General solver class.
    Helper class to Generator."""

    def solve(self, grid):
        """Returns an exhasuted grid.
        Assumes valid grid."""
        grid = grid.copy()
        while True:
            full_r = np.where(grid.all(axis=1))[0]
            full_c = np.where(grid.all(axis=0))[0]

            if self.subsequent(grid, full_r):
                continue
            if self.subsequent(grid.T, full_c):
                continue
            if self.equal_count(grid, full_r):
                continue
            if self.equal_count(grid.T, full_c):
                continue
            if self.unique(grid, full_r):
                continue
            if self.unique(grid.T, full_c):
                continue
            break

        return grid

    def subsequent(self, rows, full):
        """Checks for subsequent 1s and 2s in rows and cols.
        Returns True if any changes are made."""
        changed = False
        dim = len(rows)

        for r, row in enumerate(rows):
            if r in full:
                continue
            
            for i in range(len(row)-1):
                if row[i] == row[i+1] != 0:
                    if i < dim-2 and rows[r, i+2] == 0: # if next is empty
                        rows[r, i+2] = FLIP[row[i]]
                        changed = True

                    if i > 0 and rows[r, i-1] == 0: # if prev is empty
                        rows[r, i-1] = FLIP[row[i]]
                        changed = True
                    
                if i < len(row)-2 and row[i] == row[i+2] != 0 and row[i+1] == 0:  # Gap between two same
                    rows[r, i+1] = FLIP[row[i]]
                    changed = True

        return changed

    def equal_count(self, rows, full):
        """Checks for equal count of 1s and 2s.
        param rows: 2D array
        param full: set of rows or cols that are already full
        Returns True if any changes are made."""
        changed = False
        dim_2 = len(rows) // 2

        for r, row in enumerate(rows):
            if r in full:
                continue

            if np.sum(row == 1) == dim_2:
                rows[r, np.where(row == 0)[0]] = 2
                changed = True

            elif np.sum(row == 2) == dim_2:
                rows[r, np.where(row == 0)[0]] = 1
                changed = True
        
        return changed

    def unique(self, rows, full):
        """Checks for unique rows and cols.
        Returns True if any changes are made."""
        changed = False

        # Find rows containing exacty two gaps
        gapped_rows_indx = np.where(np.sum(rows == 0, axis=1) == 2)[0]  # [2 3]
        candidates = rows[gapped_rows_indx]  # [[2 1 0 0], [1 0 0 2]]
        full_rows = rows[full]  # [[2 1 1 2]]

        # Compare each candidate with other complete rows
        for i, candy in enumerate(candidates):
            # Find index of colored tiles
            idx = np.where(candy != 0)[0]  # [0 1]
            for row in full_rows:
                if np.array_equal(row[idx], candy[idx]):
                    # Replace the two gaps with complementary colors
                    gaps = np.where(candy == 0)[0]  # [2, 3]
                    rows[gapped_rows_indx[i], gaps] = FLIP[row[gaps]]

                    changed = True
        
        return changed
    
    def get_next(self, grid):
        """Returns the next grid in the sequence.
        Assumes valid grid."""
        grid = grid.copy()

        full_r = np.where(grid.all(axis=1))[0]
        full_c = np.where(grid.all(axis=0))[0]

        if action := self._subsequent(grid, full_r):
            return action
        if action := self._subsequent(grid.T, full_c):
            (y, x), color = action 
            return (x, y), color
        if action := self._equal_count(grid, full_r):
            return action
        if action := self._equal_count(grid.T, full_c):
            (y, x), color = action 
            return (x, y), color
        if action := self._unique(grid, full_r):
            return action
        if action := self._unique(grid.T, full_c):
            (y, x), color = action 
            return (x, y), color
        raise Exception("No action found")
        
    def _subsequent(self, rows, full):
        """Checks for subsequent 1s and 2s in rows and cols.
        Returns True if any changes are made."""
        dim = len(rows)

        for r, row in enumerate(rows):
            if r in full:
                continue
            
            for i in range(len(row)-1):
                if row[i] == row[i+1] != 0:
                    if i < dim-2 and rows[r, i+2] == 0: # if next is empty
                        return (r, i+2), FLIP[row[i]]

                    if i > 0 and rows[r, i-1] == 0: # if prev is empty
                        return (r, i-1), FLIP[row[i]]
                    
                if i < len(row)-2 and row[i] == row[i+2] != 0 and row[i+1] == 0:  # Gap between two same
                    return (r, i+1), FLIP[row[i]]

    def _equal_count(self, rows, full):
        """Checks for equal count of 1s and 2s.
        param rows: 2D array
        param full: set of rows or cols that are already full
        Returns True if any changes are made."""
        dim_2 = len(rows) // 2

        for r, row in enumerate(rows):
            if r in full:
                continue

            if np.sum(row == 1) == dim_2:
                return (r, np.where(row == 0)[0][0]), 2

            elif np.sum(row == 2) == dim_2:
                return (r, np.where(row == 0)[0][0]), 1
        
    def _unique(self, rows, full):
        """Checks for unique rows and cols.
        Returns True if any changes are made."""

        # Find rows containing exacty two gaps
        gapped_rows_indx = np.where(np.sum(rows == 0, axis=1) == 2)[0]  # [2 3]
        candidates = rows[gapped_rows_indx]  # [[2 1 0 0], [1 0 0 2]]
        full_rows = rows[full]  # [[2 1 1 2]]

        # Compare each candidate with other complete rows
        for i, candy in enumerate(candidates):
            # Find index of colored tiles
            idx = np.where(candy != 0)[0]  # [0 1]
            for row in full_rows:
                if np.array_equal(row[idx], candy[idx]):
                    # Replace the two gaps with complementary colors
                    gaps = np.where(candy == 0)[0][0]  # [2, 3]
                    return (gapped_rows_indx[i], gaps), FLIP[row[gaps]]
    

GRID = E.copy()
solver = Solver()
print(GRID)

NameError: name 'E' is not defined

In [59]:
A = np.array([4, 8, 10, 12])
print(480 // A // 12)

[10  5  4  3]


In [52]:
action = solver.get_next(GRID)
print(action)
(y, x), COLOR = action
GRID[y, x] = COLOR
print(GRID)


((3, 3), 2)
[[1 2 2 1]
 [2 1 1 2]
 [2 1 2 1]
 [1 2 1 2]]


In [352]:
solver = Solver()
GRID = np.load('grid.npy')
GRID = solver.solve(GRID)
print(GRID, valid_grid(GRID))

[[1 1 2 2 1 1 2 2]
 [2 2 1 2 1 2 1 1]
 [1 1 2 1 2 1 2 2]
 [2 2 1 1 2 2 1 1]
 [1 1 2 2 1 2 1 2]
 [2 1 1 2 1 1 2 2]
 [2 2 1 1 2 1 2 1]
 [1 2 2 1 2 2 1 1]] True


## Generator

In [349]:
class Generator():
    """Generates Takuzu puzzles."""
    # https://stackoverflow.com/questions/6924216/how-to-generate-sudoku-boards-with-unique-solutions

    def __init__(self):
        self.solver = Solver()
        self.solve = self.solver.solve
    
    def valid_grid(self, grid, verbose=False):
        """Check that grid is valid."""
        dim = len(grid)
        dim_2 = dim // 2
        
        # Equal count
        if np.any(np.sum(grid == 1, axis=1) > dim_2) or np.any(np.sum(grid == 2, axis=1) > dim_2):
            if verbose:
                print("Unequal count - rows")
            return False
        if np.any(np.sum(grid == 1, axis=0) > dim_2) or np.any(np.sum(grid == 2, axis=0) > dim_2):
            if verbose:
                print("Unequal count - cols")
            return False
        
        # Subsequent (3 in a row)
        for r in range(dim):
            for c in range(dim):
                if c < dim-2 and grid[r, c] == grid[r, c+1] == grid[r, c+2] != 0:
                    if verbose:
                        print("Subsequent - rows")
                    return False
                if r < dim-2 and grid[r, c] == grid[r+1, c] == grid[r+2, c] != 0:
                    if verbose:
                        print("Subsequent - cols")
                    return False
        
        # Unique
        full_rows = grid[np.where(grid.all(axis=1))[0]]
        if len(full_rows) != len(np.unique(full_rows, axis=0)):
            if verbose:
                print("Duplicate - rows")
            return False
        
        full_cols = grid.T[np.where(grid.all(axis=0))[0]]
        if len(full_cols) != len(np.unique(full_cols, axis=0)):
            if verbose:
                print("Duplicate - cols")
            return False
        
        return True

    def generate_grid(self, dim):
        print("Generating grid...")
        new = np.zeros((dim, dim), dtype=np.int8)
        complete = self.solve_search(new)
        return complete
        # grid = self.subtract(complete)
        return grid
    
    # Second algorithm from stackoverflow
    def solve_search(self, grid, depth=0):
        """Solves and searches for solution.
        Returns None if no solution found or grid if solution found."""
        # print(depth)
        grid = self.solve(grid)
        
        if not self.valid_grid(grid): # Invalid, so backtrack
            # print(grid,'\n')
            return None
        
        if 0 not in grid:  # Solved, so terminal state
            return grid

        # Randomly select empty cell and fill with 1 or 2
        empty_cells = list(zip(*np.where(grid == 0)))
        tile = empty_cells[np.random.randint(len(empty_cells))]
        color = np.random.randint(1, 3)

        grid[tile] = color
        result = self.solve_search(grid, depth+1)

        if result is None:
            grid[tile] = FLIP[color]
            result = self.solve_search(grid, depth+1)
        return result

    # First algorithm from stackoverflow
    def subtract(self, grid):
        """Subtracts a random tiles until one more subtraction requres solve_search."""
        
        # 2) Shuffle a list of all tiles
        tiles = [(r, c) for r in range(len(grid)) for c in range(len(grid))]
        np.random.shuffle(tiles) 

        while tiles:
            # 3) Remove a tile from grid
            tile = tiles.pop()
            color = grid[tile]
            grid[tile] = 0

            # # 4) Test uniqueness of grid
            # if not self.is_unique(grid):
            #     # 6) If not unique, add tile back and continue
            #     grid[tile] = color

            

        return grid
    
    def is_unique(self, grid):
        """Checks if grid is unique."""
        return self.unique_solution(grid) == 1
    
    def unique_solution(self, grid, solved=False):
        """Solves and searches for solution.
        Returns False if multiple solutions."""

        grid = self.solve(grid)
        
        if not self.valid_grid(grid): # Invalid, so backtrack
            return False
        
        if 0 not in grid:  # Solved, so terminal state
            return 1

        # Find frist instance of 0
        zeros = np.where(grid == 0)
        tile = zeros[0][0], zeros[1][0]

        # First go left, then right
        grid[tile] = 1
        left = self.unique_solution(grid, solved)

        grid[tile] = 2
        right = self.unique_solution(grid, solved)

        if left == 2 or right == 2:
            return 2
        if left == 1 and right == 1:
            return 2
        return self.unique_solution(grid, solved)

generator = Generator()
grid = generator.generate_grid(4)
grid = generator.subtract(grid)
# grid = np.zeros((4, 4), dtype=np.int8)
# grid = np.array([[1, 2], [2, 1]])
print(grid, generator.is_unique(grid))


Generating grid...
[[2 0 2 0]
 [0 0 0 1]
 [0 0 0 0]
 [0 0 2 0]] True


In [339]:
GRID[0, 0] = 
GRID

array([[0, 0, 0, 2],
       [0, 0, 0, 1],
       [2, 1, 2, 1],
       [1, 2, 1, 2]], dtype=int8)

In [249]:
A = np.zeros((4, 4), dtype=np.int8)
A[0, 0] = 1
A[0, 1] = 1
# Find firs tinstance of 0
zeros = np.where(A == 0)
print(zeros)
tile = zeros[0][0], zeros[1][0]
tile

(array([0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3], dtype=int64), array([2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3], dtype=int64))


(0, 2)

In [201]:
A = [1, 2, 3 ,4]
while A:
    
    A.pop()
    print(A)

[1, 2, 3]
[1, 2]
[1]
[]


## Save and load

In [8]:
# SAVE AND LOAD configirations

# save = {str(i) : a for i, a in enumerate(configs)}  # Give them names 
configs = [[1, 0], A, B]  # Or just use a list
np.savez("duko_configs.npz", *configs)  # Save them

loaded = np.load("duko_configs.npz")
configs = loaded.values()
for kek in configs:
    print(kek)

[1 0]
[[1 1 2 2]
 [2 1 1 2]
 [2 2 1 1]
 [1 2 2 1]]
[[1 1 2 2]
 [2 2 1 1]
 [2 2 1 1]
 [1 2 2 1]]


In [9]:
# np.save('grid.npy', save)  # Save grid

# loaded = np.load('grid.npy')
# loaded